In [19]:
year = 2022

In [20]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from selenium import webdriver

In [21]:
driver = webdriver.Chrome(executable_path = "/Users/julia/Downloads/chromedriver_win32/chromedriver.exe")

warning

In [22]:
import time

player_stats_url = "https://www.basketball-reference.com/leagues/NBA_{}_per_game.html"
url = player_stats_url.format(year)

driver.get(url)
driver.execute_script("window.scrollTo(1,10000)")
time.sleep(2)

html = driver.page_source
with open("NBA Stats 1991-Present/{}.html".format(year), "w+", encoding = "utf-8") as f:
    f.write(html)

In [23]:
with open("NBA Stats 1991-Present/{}.html".format(year), encoding="utf-8") as f:
    page = f.read()

    soup = BeautifulSoup(page, "html.parser")
    soup.find('tr', class_= "thead").decompose()
    player_table = soup.find(id = "per_game_stats")
    player = pd.read_html(str(player_table))[0]
    player["Year"] = year
    player.to_csv("NBA Stats 1991-Present/{}_nba_players.csv".format(year))


In [24]:
def single_row(df):
    if df.shape[0] == 1:
        return df
    else:
        row = df[df["Tm"] == "TOT"]
        row["Tm"] = df.iloc[-1,:]["Tm"]
        return row

In [25]:
players = pd.read_csv("NBA Stats 1991-Present/{}_nba_players.csv".format(year))
del players["Unnamed: 0"]
del players["Rk"]
players = players.groupby(["Player", "Year"]).apply(single_row)
players.index = players.index.droplevel()
players.index = players.index.droplevel()
players.to_csv("NBA Stats 1991-Present/{}_nba_players.csv".format(year))
